In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
!pip install omegaconf

Defaulting to user installation because normal site-packages is not writeable


In [79]:
import sys
import os
repo_dir = '/media/user/repos/bulk-euclid-cutouts'
os.path.isdir(repo_dir)
sys.path.insert(0,repo_dir)

In [80]:
import logging
logging.basicConfig(level=logging.INFO)

from omegaconf import OmegaConf
import pandas as pd

from bulk_euclid.utils import pipeline_utils
from bulk_euclid.external_targets import pipeline

In [81]:
from astroquery.esa.euclid.core import Euclid
Euclid.login()

INFO:astroquery:Login to Euclid TAP server


INFO: Login to Euclid TAP server [astroquery.esa.euclid.core]


User:  mwalms01
Password:  ········


INFO:astroquery:OK
INFO:astroquery:Login to Euclid data service
INFO:astroquery:OK
INFO:astroquery:Login to Euclid cutout service


INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid data service [astroquery.esa.euclid.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to Euclid cutout service [astroquery.esa.euclid.core]


INFO:astroquery:OK


INFO: OK [astroquery.utils.tap.core]


In [82]:
# configuration: what to download
cfg_dict = dict(
    tile_dir='/media/user/repos/bulk-euclid-cutouts/tests/test_downloading',
    release_name='F-006',
    bands=['VIS', 'NIR_Y'],
    auxillary_products=['MERPSF', 'MERRMS', 'MERBKG']
)
cfg = OmegaConf.create(cfg_dict)  # easy to access with cfg.base_dir, etc

In [83]:
tile_index = 102021539  # in the full pipeline, we iterate through each tile index, so it isn't part of cfg

In [84]:
flux_tile_metadata = pipeline_utils.get_tiles_in_survey(tile_index=tile_index, bands=cfg.bands, release_name=cfg.release_name)
flux_tile_metadata

INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
Found 2  query results


,category,checksum,creation_date,crpix1,crpix2,crval1,crval2,data_release_oid,data_size,dec,...,ra,release_date,release_name,second_type,stc_s,technique,tile_index,to_be_published,zero_point,zero_point_error
0,SCIENCE,cd0274bbfee8095dfea81d0943240651,2024-04-06T12:31:47.458,9600.0,9600.0,92.511702,-48.0,9,-32.0,-48.0,...,92.511702,<NA>,F-006,SKY,Polygon J2000 92.91225140887455 -48.2659414530...,IMAGE,102021539,1,29.8,0.1
1,SCIENCE,8c04a2f6402477d88a04b8f98e1e2337,2024-04-06T12:07:04.858,9600.0,9600.0,92.511702,-48.0,9,-32.0,-48.0,...,92.511702,<NA>,F-006,SKY,Polygon J2000 92.91225140887455 -48.2659414530...,IMAGE,102021539,1,24.6,0.1


In [85]:
flux_tile = flux_tile_metadata.iloc[1]
auxillary_tile_metadata = pipeline_utils.get_auxillary_tiles(flux_tile['mosaic_product_oid']) 
auxillary_tile_metadata

,aux_mosaic_oid,checksum,file_name,file_path,mosaic_product_oid,product_type_sas,stype,to_be_published
0,44741,57e84f6d45aec9a82a2d81afa90d2049,EUC_MER_BGMOD-VIS_TILE102021539-8FE24F_2024040...,/data_staging_otf/repository_otf/F-006/MER/102...,9965,MERBKG,BKG,1
1,44744,789b3736bb3b50dd520fcfd1b5c7493f,EUC_MER_MOSAIC-VIS-RMS_TILE102021539-751A38_20...,/data_staging_otf/repository_otf/F-006/MER/102...,9965,MERRMS,RMS,1
2,44743,69309f1fd9f15dd2da69c926095e0e92,EUC_MER_CATALOG-PSF-VIS_TILE102021539-71CC48_2...,/data_staging_otf/repository_otf/F-006/MER/102...,9965,MERPSF,PSF,1


In [86]:
dict_of_locs = pipeline.download_all_data_at_tile_index(cfg, tile_index)
dict_of_locs

INFO:astroquery:Query finished.
INFO:root:Downloaded flux+auxillary tiles: {'NIR_Y': {'FLUX': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE102021539-677258_20240405T223525.219021Z_00.00.fits', 'MERBKG': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGMOD-NIR-Y_TILE102021539-F630D_20240405T223525.218832Z_00.00.fits', 'MERPSF': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_CATALOG-PSF-NIR-Y_TILE102021539-989050_20240406T034820.708751Z_00.00.fits', 'MERRMS': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_MOSAIC-NIR-Y-RMS_TILE102021539-F9D4BF_20240405T222015.990175Z_00.00.fits'}, 'VIS': {'FLUX': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-VIS_TILE102021539-F91316_20240405T233444.877933Z_00.00.fits', 'MERBKG': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGMOD-VIS_TILE102021539-8FE24F_20240405T233444.877721Z_00

INFO: Query finished. [astroquery.utils.tap.core]
Found 2  query results


{'NIR_Y': {'FLUX': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-NIR-Y_TILE102021539-677258_20240405T223525.219021Z_00.00.fits',
  'MERBKG': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGMOD-NIR-Y_TILE102021539-F630D_20240405T223525.218832Z_00.00.fits',
  'MERPSF': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_CATALOG-PSF-NIR-Y_TILE102021539-989050_20240406T034820.708751Z_00.00.fits',
  'MERRMS': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_MOSAIC-NIR-Y-RMS_TILE102021539-F9D4BF_20240405T222015.990175Z_00.00.fits'},
 'VIS': {'FLUX': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGSUB-MOSAIC-VIS_TILE102021539-F91316_20240405T233444.877933Z_00.00.fits',
  'MERBKG': '/media/user/repos/bulk-euclid-cutouts/tests/test_downloading/EUC_MER_BGMOD-VIS_TILE102021539-8FE24F_20240405T233444.877721Z_00.00.fits',
  'MERPSF': '/media/user/repos/bulk-euclid-cutouts/test